In [36]:
import importlib
import sys
from pathlib import Path
import os
import json
import pandas as pd
import numpy as np
from scipy.stats import norm

In [2]:
project_root = Path.cwd().parents[1]
src_path = project_root / "src"
sys.path.insert(0, str(project_root))

In [3]:
from src.config import llm_config
importlib.reload(llm_config)
from src.config.llm_config import get_llm

## Test data loader

### Core level

#### csv_loader

In [11]:
from src.core import data_loader
importlib.reload(data_loader)
from src.core.data_loader import csv_loader

In [12]:
filepath = f"{project_root}/inputs/dummy_options.csv"

In [13]:
df = csv_loader.func(filepath)
df

'{"date":{"0":"2025-09-01","1":"2025-09-02","2":"2025-09-03","3":"2025-09-04","4":"2025-09-05","5":"2025-09-06","6":"2025-09-07","7":"2025-09-08","8":"2025-09-09","9":"2025-09-10"},"S":{"0":100,"1":102,"2":98,"3":101,"4":99,"5":103,"6":97,"7":100,"8":104,"9":96},"K":{"0":105,"1":106,"2":104,"3":107,"4":103,"5":108,"6":102,"7":106,"8":109,"9":101},"T":{"0":1.0,"1":0.9,"2":0.8,"3":0.7,"4":0.6,"5":0.5,"6":0.4,"7":0.3,"8":0.2,"9":0.1},"r":{"0":0.05,"1":0.045,"2":0.048,"3":0.047,"4":0.046,"5":0.049,"6":0.044,"7":0.05,"8":0.045,"9":0.048},"sigma":{"0":0.2,"1":0.19,"2":0.21,"3":0.18,"4":0.22,"5":0.2,"6":0.23,"7":0.19,"8":0.21,"9":0.2},"option_type":{"0":"call","1":"put","2":"call","3":"call","4":"put","5":"put","6":"call","7":"call","8":"put","9":"put"}}'

### Agent level

In [14]:
from src.agents.nodes import data_loader_agent
importlib.reload(data_loader_agent)
from src.agents.nodes.data_loader_agent import DataLoader

In [15]:
llm = get_llm()
agent = DataLoader(llm)

In [17]:
csv_path = os.path.join(project_root, 'inputs', 'dummy_options.csv')
csv_path = os.path.abspath(csv_path)
print(f"\nTest CSV path: {csv_path}")
print(f"File exists: {os.path.exists(csv_path)}")


Test CSV path: /Users/yifanli/Library/CloudStorage/OneDrive-EY/github/model_doc_automation/inputs/dummy_options.csv
File exists: True


In [18]:
initial_state = {
    "messages": [],
    "csv_file_path": csv_path,
    "csv_data": None,
    "data_loader_agent_status": None,
    "calculation_results": None,
    "greeks_data": None,
    "calculator_agent_status": None,
    "test_results": None,
    "test_agent_status": None,
    "summary_text": None,
    "summarty_writer_agnet_status": None,
    "charts": None,
    "chart_descriptions": None,
    "chart_generator_agent_status": None,
    "final_report_path": None,
    "final_report_html": None,
    "report_assembler_agent_status": None,
    "current_agent": None,
    "workflow_status": "started",
    "errors": []
}

In [19]:
result = agent(initial_state)

In [23]:
print("\nAgent execution completed!")
print(f"\nAgent Status: {result.get('agent_status')}")
print(f"Current Agent: {result.get('current_agent')}")
print(f"Workflow Status: {result.get('workflow_status')}")
print(f"Data Status: {result.get('csv_data')}")


Agent execution completed!

Agent Status: completed
Current Agent: data_loader
Workflow Status: in_progress
Data Status: {'date': {'0': '2025-09-01', '1': '2025-09-02', '2': '2025-09-03', '3': '2025-09-04', '4': '2025-09-05', '5': '2025-09-06', '6': '2025-09-07', '7': '2025-09-08', '8': '2025-09-09', '9': '2025-09-10'}, 'S': {'0': 100, '1': 102, '2': 98, '3': 101, '4': 99, '5': 103, '6': 97, '7': 100, '8': 104, '9': 96}, 'K': {'0': 105, '1': 106, '2': 104, '3': 107, '4': 103, '5': 108, '6': 102, '7': 106, '8': 109, '9': 101}, 'T': {'0': 1.0, '1': 0.9, '2': 0.8, '3': 0.7, '4': 0.6, '5': 0.5, '6': 0.4, '7': 0.3, '8': 0.2, '9': 0.1}, 'r': {'0': 0.05, '1': 0.045, '2': 0.048, '3': 0.047, '4': 0.046, '5': 0.049, '6': 0.044, '7': 0.05, '8': 0.045, '9': 0.048}, 'sigma': {'0': 0.2, '1': 0.19, '2': 0.21, '3': 0.18, '4': 0.22, '5': 0.2, '6': 0.23, '7': 0.19, '8': 0.21, '9': 0.2}, 'option_type': {'0': 'call', '1': 'put', '2': 'call', '3': 'call', '4': 'put', '5': 'put', '6': 'call', '7': 'call', 

## Test calculator

### Tool level

#### bsm_calculator

In [28]:
from src.core.data_loader import csv_loader

filepath = f"{project_root}/inputs/dummy_options.csv"
df = csv_loader.func(filepath)
df

'{"date":{"0":"2025-09-01","1":"2025-09-02","2":"2025-09-03","3":"2025-09-04","4":"2025-09-05","5":"2025-09-06","6":"2025-09-07","7":"2025-09-08","8":"2025-09-09","9":"2025-09-10"},"S":{"0":100,"1":102,"2":98,"3":101,"4":99,"5":103,"6":97,"7":100,"8":104,"9":96},"K":{"0":105,"1":106,"2":104,"3":107,"4":103,"5":108,"6":102,"7":106,"8":109,"9":101},"T":{"0":1.0,"1":0.9,"2":0.8,"3":0.7,"4":0.6,"5":0.5,"6":0.4,"7":0.3,"8":0.2,"9":0.1},"r":{"0":0.05,"1":0.045,"2":0.048,"3":0.047,"4":0.046,"5":0.049,"6":0.044,"7":0.05,"8":0.045,"9":0.048},"sigma":{"0":0.2,"1":0.19,"2":0.21,"3":0.18,"4":0.22,"5":0.2,"6":0.23,"7":0.19,"8":0.21,"9":0.2},"option_type":{"0":"call","1":"put","2":"call","3":"call","4":"put","5":"put","6":"call","7":"call","8":"put","9":"put"}}'

In [29]:
csv_data = df

In [30]:
[isinstance(csv_data, str),isinstance(csv_data, dict)]

[True, False]

In [34]:
data = json.loads(csv_data)
df = pd.DataFrame(data)

In [35]:
required_cols = ['option_type', 'S', 'K', 'T', 'r', 'sigma']
missing_cols = [col for col in required_cols if col not in df.columns]
missing_cols

[]

In [37]:
from src.core.bsm_calculator import bsm_calculator
def calc_row(row):
    return bsm_calculator.func(
        option_type=row['option_type'], 
        S=row['S'], 
        K=row['K'], 
        T=row['T'], 
        r=row['r'], 
        sigma=row['sigma'], 
    )

In [39]:
df['BSM_Price'] = df.apply(
    calc_row, axis=1
)


result = {
    "type": "Black-Scholes Option Pricing Results",
    "data": df.to_json(orient="records")
}
result

{'type': 'Black-Scholes Option Pricing Results',
 'data': '[{"date":"2025-09-01","S":100,"K":105,"T":1.0,"r":0.05,"sigma":0.2,"option_type":"call","BSM_Price":"8.021352235143176"},{"date":"2025-09-02","S":102,"K":106,"T":0.9,"r":0.045,"sigma":0.19,"option_type":"put","BSM_Price":"7.214238371309641"},{"date":"2025-09-03","S":98,"K":104,"T":0.8,"r":0.048,"sigma":0.21,"option_type":"call","BSM_Price":"6.415749358967169"},{"date":"2025-09-04","S":101,"K":107,"T":0.7,"r":0.047,"sigma":0.18,"option_type":"call","BSM_Price":"4.952961817553408"},{"date":"2025-09-05","S":99,"K":103,"T":0.6,"r":0.046,"sigma":0.22,"option_type":"put","BSM_Price":"7.377696042475904"},{"date":"2025-09-06","S":103,"K":108,"T":0.5,"r":0.049,"sigma":0.2,"option_type":"put","BSM_Price":"7.143363876009964"},{"date":"2025-09-07","S":97,"K":102,"T":0.4,"r":0.044,"sigma":0.23,"option_type":"call","BSM_Price":"4.250515622426249"},{"date":"2025-09-08","S":100,"K":106,"T":0.3,"r":0.05,"sigma":0.19,"option_type":"call","BSM_Pr

In [40]:
from src.core.bsm_calculator import batch_bsm_calculator

result = batch_bsm_calculator.func(data)
result

{'type': 'Black-Scholes Option Pricing Results',
 'data': '[{"date":"2025-09-01","S":100,"K":105,"T":1.0,"r":0.05,"sigma":0.2,"option_type":"call","BSM_Price":"8.021352235143176"},{"date":"2025-09-02","S":102,"K":106,"T":0.9,"r":0.045,"sigma":0.19,"option_type":"put","BSM_Price":"7.214238371309641"},{"date":"2025-09-03","S":98,"K":104,"T":0.8,"r":0.048,"sigma":0.21,"option_type":"call","BSM_Price":"6.415749358967169"},{"date":"2025-09-04","S":101,"K":107,"T":0.7,"r":0.047,"sigma":0.18,"option_type":"call","BSM_Price":"4.952961817553408"},{"date":"2025-09-05","S":99,"K":103,"T":0.6,"r":0.046,"sigma":0.22,"option_type":"put","BSM_Price":"7.377696042475904"},{"date":"2025-09-06","S":103,"K":108,"T":0.5,"r":0.049,"sigma":0.2,"option_type":"put","BSM_Price":"7.143363876009964"},{"date":"2025-09-07","S":97,"K":102,"T":0.4,"r":0.044,"sigma":0.23,"option_type":"call","BSM_Price":"4.250515622426249"},{"date":"2025-09-08","S":100,"K":106,"T":0.3,"r":0.05,"sigma":0.19,"option_type":"call","BSM_Pr

### Agent level

In [52]:
from src.agents.nodes import calculator_agent
importlib.reload(calculator_agent)
from src.agents.nodes.calculator_agent import Calculator

In [53]:
llm = get_llm()
agent = Calculator(llm)

In [54]:
filepath = f"{project_root}/inputs/dummy_options.csv"
mock_csv_data = csv_loader.func(filepath)

In [55]:
initial_state = {
    "messages": [],
    "csv_file_path": "/inputs/dummy_options.csv",
    "csv_data": mock_csv_data,
    "data_loader_agent_status": "completed",
    "calculation_results": None,
    "greeks_data": None,
    "calculator_agent_status": None,
    "test_results": None,
    "test_agent_status": None,
    "summary_text": None,
    "summarty_writer_agnet_status": None,
    "charts": None,
    "chart_descriptions": None,
    "chart_generator_agent_status": None,
    "final_report_path": None,
    "final_report_html": None,
    "report_assembler_agent_status": None,
    "current_agent": "data_loader",
    "workflow_status": "in_progress",
    "errors": []
}


In [ ]:
result = agent(initial_state)

print("\nAgent execution completed!")
print(f"\nAgent Status: {result.get('calculator_agent_status')}")
print(f"Current Agent: {result.get('current_agent')}")
print(f"Workflow Status: {result.get('workflow_status')}")
print(f"Data Result: {result.get('csv_data')}")


Agent execution completed!

Agent Status: completed
Current Agent: calculator
Workflow Status: in_progress
Data Status: None


## Test testor

### test_batch_greeks_validator

In [26]:
from src.core.bsm_validator import batch_greeks_validator

In [6]:
print("\n" + "="*70)
print("TEST 1: Batch Greeks Validator")
print("="*70)


TEST 1: Batch Greeks Validator


In [7]:
sample_data = {
    "option_type": {0: "call", 1: "put", 2: "call"},
    "S": {0: 100.0, 1: 100.0, 2: 105.0},
    "K": {0: 100.0, 1: 100.0, 2: 100.0},
    "T": {0: 1.0, 1: 1.0, 2: 0.5},
    "r": {0: 0.05, 1: 0.05, 2: 0.05},
    "sigma": {0: 0.2, 1: 0.2, 2: 0.25}
}

#### Step 1: _parse_csv_data

In [8]:
csv_data = sample_data

In [10]:
[isinstance(csv_data, str),isinstance(csv_data, dict)]

[False, True]

In [18]:
data = csv_data
df = pd.DataFrame(data)

#### Step 2

In [20]:
required_cols = ['option_type', 'S', 'K', 'T', 'r', 'sigma']
missing = [c for c in required_cols if c not in df.columns]
missing

[]

In [21]:
results = {
    "total_options": len(df),
    "passed": 0,
    "failed": 0,
    "errors": 0,
    "details": []
}

In [22]:
idx, row = list(df.iterrows())[0]

In [24]:
option_type = str(row['option_type']).lower()
S, K, T, r, sigma = float(row['S']), float(row['K']), float(row['T']), float(row['r']), float(row['sigma'])

In [27]:
result_json = batch_greeks_validator.invoke({"csv_data": sample_data})
result = json.loads(result_json)

In [28]:
print("\n" + "-"*70)
print("Validation Results:")
print("-"*70)
print(f"Status: {result['status']}")
print(f"Total Options: {result['total_options']}")
print(f"Passed: {result['passed']}")
print(f"Failed: {result['failed']}")
print(f"Errors: {result['errors']}")


----------------------------------------------------------------------
Validation Results:
----------------------------------------------------------------------
Status: passed
Total Options: 3
Passed: 3
Failed: 0
Errors: 0


### test_validate_put_call_parity

In [31]:
from src.core.bsm_validator import validate_put_call_parity

In [32]:
print("\n" + "="*70)
print("TEST 2: Put-Call Parity Validator")
print("="*70)


TEST 2: Put-Call Parity Validator


In [33]:
sample_data = {
    "option_type": {0: "call", 1: "put", 2: "call", 3: "put"},
    "S": {0: 100.0, 1: 100.0, 2: 105.0, 3: 105.0},
    "K": {0: 100.0, 1: 100.0, 2: 100.0, 3: 100.0},
    "T": {0: 1.0, 1: 1.0, 2: 0.5, 3: 0.5},
    "r": {0: 0.05, 1: 0.05, 2: 0.05, 3: 0.05},
    "sigma": {0: 0.2, 1: 0.2, 2: 0.25, 3: 0.25}
}

In [34]:
result_json = validate_put_call_parity.invoke({"csv_data": sample_data})
result = json.loads(result_json)

print("\n" + "-"*70)
print("Parity Test Results:")
print("-"*70)
print(f"Status: {result['status']}")
print(f"Pairs Found: {result['pairs_found']}")
print(f"Pairs Tested: {result['pairs_tested']}")
print(f"Pairs Passed: {result['pairs_passed']}")
print(f"Pairs Failed: {result['pairs_failed']}")


----------------------------------------------------------------------
Parity Test Results:
----------------------------------------------------------------------
Status: passed
Pairs Found: 2
Pairs Tested: 2
Pairs Passed: 2
Pairs Failed: 0


### test_validate_sensitivity

In [35]:
from src.core.bsm_validator import validate_sensitivity

In [36]:
print("\n" + "="*70)
print("TEST 3: Sensitivity Validator")
print("="*70)


TEST 3: Sensitivity Validator


In [37]:
sample_data = {
    "option_type": {0: "call"},
    "S": {0: 100.0},
    "K": {0: 100.0},
    "T": {0: 1.0},
    "r": {0: 0.05},
    "sigma": {0: 0.2}
}

In [38]:
result_json = validate_sensitivity.invoke({"csv_data": sample_data})
result = json.loads(result_json)

print("\n" + "-"*70)
print("Sensitivity Test Results:")
print("-"*70)
print(f"Status: {result['status']}")


----------------------------------------------------------------------
Sensitivity Test Results:
----------------------------------------------------------------------
Status: passed


### Test testor

In [37]:
from src.agents.nodes import tester_agent
importlib.reload(tester_agent)
from src.agents.nodes.tester_agent import Tester

In [38]:
print("\n" + "="*60)
print("Testing Agent 3: Tester (Test Agent)")
print("="*60)


Testing Agent 3: Tester (Test Agent)


In [41]:
llm = get_llm()
agent = Tester(llm)

In [ ]:
mock_csv_data = {
    "option_type": {0: "call", 1: "put", 2: "call", 3: "put"},
    "S": {0: 100.0, 1: 100.0, 2: 105.0, 3: 105.0},
    "K": {0: 100.0, 1: 100.0, 2: 100.0, 3: 100.0},
    "T": {0: 1.0, 1: 1.0, 2: 0.5, 3: 0.5},
    "r": {0: 0.05, 1: 0.05, 2: 0.05, 3: 0.05},
    "sigma": {0: 0.2, 1: 0.2, 2: 0.25, 3: 0.25}
}

# mock_calculation_results = """
# |   K |   S |   T | option_type   |    r |   sigma |   BSM_Price |
# |----:|----:|----:|:--------------|-----:|--------:|------------:|
# | 100 | 100 | 1   | call          | 0.05 |    0.2  |    10.4506  |
# | 100 | 100 | 1   | put           | 0.05 |    0.2  |     5.57353 |
# | 100 | 105 | 0.5 | call          | 0.05 |    0.25 |    11.4774  |
# | 100 | 105 | 0.5 | put           | 0.05 |    0.25 |     4.00839 |
# """


In [ ]:
initial_state = {
    "messages": [],
    "csv_file_path": "/inputs/dummy_options.csv",
    "csv_data": mock_csv_data,
    "data_loader_agent_status": "completed",
    # "calculation_results": mock_calculation_results,
    "calculation_results": None,
    "greeks_data": None,
    "sensitivity_data": None,
    "calculator_agent_status": "completed",
    "test_results": None,
    "tester_agent_status": None,
    "summary_text": None,
    "summarty_writer_agnet_status": None,
    "charts": None,
    "chart_descriptions": None,
    "chart_generator_agent_status": None,
    "final_report_path": None,
    "final_report_html": None,
    "report_assembler_agent_status": None,
    "current_agent": "agent2",
    "workflow_status": "in_progress",
    "errors": []
}
print("\n" + "-"*60)
print("Running Test Agent...")
print("-"*60)
print("\nThis agent will execute actual pytest test functions:")
print("  - test_greeks_call_basic")
print("  - test_greeks_put_call_parity")
print("  - test_sensitivity_length_and_fields")


------------------------------------------------------------
Running Test Agent...
------------------------------------------------------------

This agent will execute actual pytest test functions:
  - test_greeks_call_basic
  - test_greeks_put_call_parity
  - test_sensitivity_length_and_fields


In [44]:
result = agent(initial_state)

print("\nAgent execution completed!")
print(f"\nAgent Status: {result.get('tester_agent_status')}")
print(f"Current Agent: {result.get('current_agent')}")
print(f"Workflow Status: {result.get('workflow_status')}")


Agent execution completed!

Agent Status: completed
Current Agent: tester
Workflow Status: in_progress


## Test summary writer